# Errors and Exceptions

No matter your skill as a programmer, you will eventually make a coding mistake.
Such mistakes come in three basic flavors:

- *Syntax errors:* Errors where the code is not valid Python (generally easy to fix)
- *Runtime errors:* Errors where syntactically valid code fails to execute, perhaps due to invalid user input (sometimes easy to fix)
- *Semantic errors:* Errors in logic: code executes without a problem, but the result is not what you expect (often very difficult to track-down and fix)

Here we're going to focus on how to deal cleanly with *runtime errors*.
As we'll see, Python handles runtime errors via its *exception handling* framework.

## Runtime Errors

If you've done any coding in Python, you've likely come across runtime errors.
They can happen in a lot of ways.

For example, if you try to reference an undefined variable:

In [3]:
print(Q)

NameError: name 'Q' is not defined

Or if you try an operation that's not defined:

In [4]:
1 + 'abc'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

Or you might be trying to compute a mathematically ill-defined result:

In [5]:
2 / 0

ZeroDivisionError: division by zero

Or maybe you're trying to access a sequence element that doesn't exist:

In [6]:
L = [1, 2, 3]
L[1000]

IndexError: list index out of range

Note that in each case, Python is kind enough to not simply indicate that an error happened, but to spit out a *meaningful* exception that includes information about what exactly went wrong, along with the exact line of code where the error happened.
Having access to meaningful errors like this is immensely useful when trying to trace the root of problems in your code.

## Catching Exceptions: ``try`` and ``except``
The main tool Python gives you for handling runtime exceptions is the ``try``...``except`` clause.
Its basic structure is this:

In [7]:
try:
    print("this gets executed first")
except:
    print("this gets executed only if there is an error")

this gets executed first


Note that the second block here did not get executed: this is because the first block did not return an error.
Let's put a problematic statement in the ``try`` block and see what happens:

In [9]:
try:
    print("let's try something:")
    x = 1 / 0 # ZeroDivisionError
    print("something happened!")
except:
    print("something bad happened!")

let's try something:
something bad happened!


Here we see that when the error was raised in the ``try`` statement (in this case, a ``ZeroDivisionError``), the error was caught, and the ``except`` statement was executed.

One way this is often used is to check user input within a function or another piece of code.
For example, we might wish to have a function that catches zero-division and returns some other value, perhaps a suitably large number like $10^{100}$:

In [10]:
def safe_divide(a, b):
    try:
        return a / b
    except:
        return "error"

In [11]:
safe_divide(1, 2)

0.5

In [12]:
safe_divide(2, 0)

'error'

There is a subtle problem with this code, though: what happens when another type of exception comes up? For example, this is probably not what we intended:

In [13]:
safe_divide (1, '2')

'error'

Dividing an integer and a string raises a ``TypeError``, which our over-zealous code caught and assumed was a ``ZeroDivisionError``!
For this reason, it's nearly always a better idea to catch exceptions *explicitly*:

In [27]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100
    except TypeError:
        print(a,b)
        safe_divide(1,2):
            print("safe callede again")
        return "String found"

SyntaxError: invalid syntax (<ipython-input-27-23a9447846fc>, line 8)

In [18]:
safe_divide(1, 0)

1e+100

In [20]:
safe_divide(1, "2")

0.5

We're now catching zero-division errors only, and letting all other errors pass through un-modified.

## Raising Exceptions: ``raise``
We've seen how valuable it is to have informative exceptions when using parts of the Python language.
It's equally valuable to make use of informative exceptions within the code you write, so that users of your code (foremost yourself!) can figure out what caused their errors.

The way you raise your own exceptions is with the ``raise`` statement. For example:

In [28]:
raise RuntimeError("my error message")

RuntimeError: my error message

As an example of where this might be useful, let's return to our ``fibonacci`` function that we defined previously:

In [17]:
def fibonacci(N):
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

One potential problem here is that the input value could be negative.
This will not currently cause any error in our function, but we might want to let the user know that a negative ``N`` is not supported.
Errors stemming from invalid parameter values, by convention, lead to a ``ValueError`` being raised:

In [29]:
def fibonacci(N):
    if N < 0:
        raise ValueError("N must be non-negative")
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [30]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [20]:
fibonacci(-10)

ValueError: N must be non-negative

Now the user knows exactly why the input is invalid, and could even use a ``try``...``except`` block to handle it!

In [31]:
N = -10
try:
    print("trying this...")
    print(fibonacci(N))
except ValueError:
    N *=-1
    print(fibonacci(N))
    print("Bad value: need to do something else")

trying this...
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
Bad value: need to do something else


## Diving Deeper into Exceptions

Briefly, I want to mention here some other concepts you might run into.
I'll not go into detail on these concepts and how and why to use them, but instead simply show you the syntax so you can explore more on your own.

### Accessing the error message

Sometimes in a ``try``...``except`` statement, you would like to be able to work with the error message itself.
This can be done with the ``as`` keyword:

In [32]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("Error class is:  ", type(err))
    print("Error message is:", err)

Error class is:   <class 'ZeroDivisionError'>
Error message is: division by zero


With this pattern, you can further customize the exception handling of your function.

### Defining custom exceptions
In addition to built-in exceptions, it is possible to define custom exceptions through *class inheritance*.
For instance, if you want a special kind of ``ValueError``, you can do this:

In [23]:
class MySpecialError(ValueError):
    pass

raise MySpecialError("here's the message")

MySpecialError: here's the message

This would allow you to use a ``try``...``except`` block that only catches this type of error:

In [24]:
try:
    print("do something")
    raise MySpecialError("[informative error message here]")
except MySpecialError:
    print("do something else")

do something
do something else


You might find this useful as you develop more customized code.

## ``try``...``except``...``else``...``finally``
In addition to ``try`` and ``except``, you can use the ``else`` and ``finally`` keywords to further tune your code's handling of exceptions.
The basic structure is this:

In [33]:
try:
    print("try something here")
    1/0
except:
    print("this happens only if it fails")
else:
    print("this happens only if it succeeds")
finally:
    print("this happens no matter what")

try something here
this happens only if it fails
this happens no matter what


The utility of ``else`` here is clear, but what's the point of ``finally``?
Well, the ``finally`` clause really is executed *no matter what*: I usually see it used to do some sort of cleanup after an operation completes.

In [40]:
# import module sys to get the type of exception
import sys

randomList = ['a', 0, 2]

for entry in randomList:
    try:
        print("The entry is", entry)
        r = 1/int(entry)
        break
    except:
        print("Oops!",sys.exc_info(),"occured.")
        print("Next entry.")
        print()
print("The reciprocal of",entry,"is",r)

The entry is a
Oops! (<class 'ValueError'>, ValueError("invalid literal for int() with base 10: 'a'",), <traceback object at 0x00000142F03FCEC8>) occured.
Next entry.

The entry is 0
Oops! (<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero',), <traceback object at 0x00000142F03FCEC8>) occured.
Next entry.

The entry is 2
The reciprocal of 2 is 0.5


In [ ]:
try:
   # do something
   pass

except ValueError:
   # handle ValueError exception
   pass

except (TypeError, ZeroDivisionError):
   # handle multiple exceptions
   # TypeError and ZeroDivisionError
   pass

except:
   # handle all other exceptions
   pass

In [41]:
try:
    a = int(input("Enter a positive integer: "))
    if a <= 0:
        raise ValueError("That is not a positive number!")
except ValueError as ve:
    print(ve)


Enter a positive integer: -1
That is not a positive number!


In [5]:
# define Python user-defined exceptions
class Error(Exception):
   """Base class for other exceptions"""
   pass

class ValueTooSmallError(Error):
   """Raised when the input value is too small"""
   pass

class ValueTooLargeError(Error):
   """Raised when the input value is too large"""
   pass

# our main program
# user guesses a number until he/she gets it right

# you need to guess this number
number = 10

while True:
   try:
       i_num = int(input("Enter a number: "))
       if i_num < number:
           raise ValueTooSmallError
       elif i_num > number:
           raise ValueTooLargeError
       break
   except ValueTooSmallError:
       print("This value is too small, try again!")
       print()
   except ValueTooLargeError:
       print("This value is too large, try again!")
       print()

print("Congratulations! You guessed it correctly.")

Enter a number: 2
This value is too small, try again!

Enter a number: 2
This value is too small, try again!

Enter a number: 3
This value is too small, try again!

Enter a number: 4
This value is too small, try again!

Enter a number: 1
This value is too small, try again!

Enter a number: 10
Congratulations! You guessed it correctly.


In [ ]:
BUILTIN EXCEPTION 

AssertionError	Raised when assert statement fails.

AttributeError	Raised when attribute assignment or reference fails.

EOFError	Raised when the input() functions hits end-of-file condition.

FloatingPointError	Raised when a floating point operation fails.

GeneratorExit	Raise when a generator's close() method is called.

ImportError	Raised when the imported module is not found.

IndexError	Raised when index of a sequence is out of range.

KeyError	Raised when a key is not found in a dictionary.

KeyboardInterrupt	Raised when the user hits interrupt key (Ctrl+c or delete).

MemoryError	Raised when an operation runs out of memory.

NameError	Raised when a variable is not found in local or global scope.

NotImplementedError	Raised by abstract methods.

OSError	Raised when system operation causes system related error.

OverflowError	Raised when result of an arithmetic operation is too large to be represented.

ReferenceError	Raised when a weak reference proxy is used to access a garbage collected referent.

RuntimeError	Raised when an error does not fall under any other category.

StopIteration	Raised by next() function to indicate that there is no further item to be returned by iterator.

SyntaxError	Raised by parser when syntax error is encountered.

IndentationError	Raised when there is incorrect indentation.

TabError	Raised when indentation consists of inconsistent tabs and spaces.

SystemError	Raised when interpreter detects internal error.

SystemExit	Raised by sys.exit() function.

TypeError	Raised when a function or operation is applied to an object of incorrect type.

UnboundLocalError	Raised when a reference is made to a local variable in a function or 
method, but no value has been bound to that variable.

UnicodeError	Raised when a Unicode-related encoding or decoding error occurs.

UnicodeEncodeError	Raised when a Unicode-related error occurs during encoding.

UnicodeDecodeError	Raised when a Unicode-related error occurs during decoding.

UnicodeTranslateError	Raised when a Unicode-related error occurs during translating.

ValueError	Raised when a function gets argument of correct type but improper value.

ZeroDivisionError	Raised when second operand of division or modulo operation is zero.


In [6]:
l = [1,2,3,4]
l[123]



IndexError: list index out of range

In [8]:
x = 13 
y = 0 
l = [1,2,3,4]
try:
    #x/y
    k
except ZeroDivisionError :
    print("Zero division error")
except nameError :
    print("Zero division error")

NameError: name 'nameError' is not defined

In [1]:
import datetime
import dis

from gw_utility.book import Book
from gw_utility.logging import Logging


def main():
    try:
        # Create Book.
        book = Book("The Hobbit", "J.R.R. Tolkien", 366, datetime.date(1937, 9, 15))

        # Log book object.
        Logging.line_separator("log_object(book)", 60)
        log_object(book)

        # Log invalid object.
        Logging.line_separator("log_invalid_object(book)", 60)
        log_invalid_object(book)

        # Disassemble both log_ functions.
        Logging.line_separator("DISASSEMBLY OF log_object()", 60)
        disassemble_object(log_object)

        Logging.line_separator("DISASSEMBLY OF log_invalid_object()", 60)
        disassemble_object(log_invalid_object)
    except NameError as error:
        # Output expected NameErrors.
        Logging.log_exception(error)
    except Exception as exception:
        # Output unexpected Exceptions.
        Logging.log_exception(exception, False)


def log_object(value):
    """Logs passed value parameter to console.

    :param value: Value to be logged.
    :return: None
    """
    try:
        Logging.log(value)
    except NameError as error:
        # Output expected NameErrors.
        Logging.log_exception(error)
    except Exception as exception:
        # Output unexpected Exceptions.
        Logging.log_exception(exception, False)


def log_invalid_object(value):
    """Attempts to log invalid object (valu) to console.

    :param value: Value intended to be logged, but which is instead ignored.
    :return: None
    """
    try:
        Logging.log(valu)
    except NameError as error:
        # Output expected NameErrors.
        Logging.log_exception(error)
    except Exception as exception:
        # Output unexpected Exceptions.
        Logging.log_exception(exception, False)


def disassemble_object(value):
    """Outputs disassembly of passed object.

    :param value: Object to be disassembled.
    :return: None
    """
    dis.dis(value)


if __name__ == "__main__":
    main()



ModuleNotFoundError: No module named 'gw_utility'

In [3]:
import sys
print(sys.platform)
assert ('linux' in sys.platform), "This code runs on Linux only."

win32


AssertionError: This code runs on Linux only.

In [7]:
import time

print('Press ENTER to begin, Press Ctrl + C to stop')
while True:
    try:
        input() # For ENTER. Use raw_input() if you are running python 2.x instead of input()
        starttime = time.time()
        print('Started')
        while True:
            print('Time Elapsed: ', round(time.time() - starttime, 0), 'secs', end="\r")
            time.sleep(1)
    except KeyboardInterrupt:
        print('Stopped')
        endtime = time.time()
        print('Total Time:', round(endtime - starttime, 2),'secs')
        break

Press ENTER to begin, Press Ctrl + C to stop

Started
Stoppedapsed:  1480.0 secs
Total Time: 1481.03 secs


<!--NAVIGATION-->
< [Defining and Using Functions](08-Defining-Functions.ipynb) | [Contents](Index.ipynb) | [Iterators](10-Iterators.ipynb) >

In [1]:
#User-defined exceptions
class Error(Exception):
   """Base class for other exceptions"""
   pass

#Define class for NegativeValueError
class NegativeValueError(Error):
  """Raised when the input is negative"""
  pass

#Define class for ValueTooSmallError
class ValueTooSmallError(Error):
   """Raised when the value is too small"""
   pass

#Define class for ValueTooLargeError
class ValueTooLargeError(Error):
   """Raised when the value is too large"""
   pass

#main program
#Takes input till the user inputs correct value

#Stored number
number = 11

while True:
   try:
       num = int(input("Enter a number: "))
       if num < 0:
           raise NegativeValueError
       elif num < number:
           raise ValueTooSmallError
       elif num > number:
           raise ValueTooLargeError
       break
   except NegativeValueError:
       print("This is a negative value, try again")
       print("")
   except ValueTooSmallError:
       print("This value is too small, try again")
       print("")
   except ValueTooLargeError:
       print("This value is too large, try again!")
       print("")

print("Correct value entered")

Enter a number: -1
This is a negative value, try again

Enter a number: 45
This value is too large, try again!

Enter a number: 11
Correct value entered
